In [2]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import urllib

# ---------- CONFIG ----------
API_URL = "http://localhost:5000/products"   # endpoint local (Flask)
# SQL Server connection (trusted)
driver = "ODBC Driver 17 for SQL Server"
server = r"localhost\SQLEXPRESS"  
database = "retail_project"

# Construir cadena ODBC segura
odbc_str = (
    "DRIVER={%s};SERVER=%s;DATABASE=%s;Trusted_Connection=yes;" %
    (driver, server, database)
)
connect_str = "mssql+pyodbc:///?odbc_connect=" + urllib.parse.quote_plus(odbc_str)

# ---------- GET from local API ----------
resp = requests.get(API_URL, timeout=20)
resp.raise_for_status()
products = resp.json()
df = pd.DataFrame(products)
print("Read from local API:", len(df))
print(df.head())

# ---------- Connect & load to SQL Server ----------
engine = create_engine(connect_str)

# Normalizar/renombrar columnas si hace falta
# Por ejemplo, si tu JSON usa 'product_id' y quieres 'product_id' en SQL:
# df = df[['product_id','category','price']]  # ajusta columnas disponibles

df.to_sql("products_raw", engine, if_exists="replace", index=False)
print("Table 'products_raw' filled at SQL Server.")

Read from local API: 109
   id product_category product_id  product_price
0   1      Motorcycles   S10_1678           95.0
1   2     Classic Cars   S10_1949          214.0
2   3      Motorcycles   S10_2016          118.0
3   4      Motorcycles   S10_4698          193.0
4   5     Classic Cars   S10_4757          136.0
Table 'products_raw' filled at SQL Server.
